## Heartbeat and Blox Oxygen Sensor

Details about the sensor can be found [here](https://wiki.dfrobot.com/SKU_SEN0518_Gravity_MAX30102_Heart_Rate_and_Oximeter_Sensor). 
For the Notebooks to work, the driver Python files `DFRobot_BloodOxygen_S.py` and `DFRobot_RTU.py` need to be in the same folder.

In [ ]:
import sys
import os
import time
import RPi.GPIO as GPIO
from IPython.display import clear_output, display
from DFRobot_BloodOxygen_S import *
from datetime import datetime
from matplotlib import pyplot

I2C_1       = 0x01      
I2C_ADDRESS = 0x57

max30102 = DFRobot_BloodOxygen_S_i2c(I2C_1,I2C_ADDRESS)

%matplotlib inline
pyplot.rcParams['figure.figsize'] = [10, 5]

In [ ]:
# Setup
while (False == max30102.begin()):
    print("init fail!")
    time.sleep(1)

#### Adding the LCD screen

Using the code from the Enivro Pi camp, we create a class to write to the display.

In [ ]:
import ST7735
from PIL import Image, ImageDraw, ImageFont
from fonts.ttf import RobotoMedium as UserFont

class lcdscreen():
    def __init__(self):
        self.disp = ST7735.ST7735(port=0,cs=1,dc=9,backlight=12,rotation=270,spi_speed_hz=10000000)
        self.disp.begin()

        self.back_color = (0, 60, 20)
        self.text_color = (255, 255, 255)
        self.head_color = (255, 255,   0)
        self.warn_color = (255,  50,  50)
        self.font_size  = 14
        self.font_size_big = 24
        self.text_margin= 5

        self.img = Image.new('RGBA', (self.disp.width, self.disp.height))
        self.draw = ImageDraw.Draw(self.img)
        self.font = ImageFont.truetype(UserFont, self.font_size)
        self.font_big = ImageFont.truetype(UserFont, self.font_size_big)
        self.size_x, self.size_y = self.draw.textsize('text', self.font)
        
        self.header = "ORCSPIHealth Station"
        self.label = "Waiting"
        self.value = ""
        self.update(self.label, self.value)

    def update(self, label, value): 
        self.label = label;
        self.value = value
        
        self.draw.rectangle((0, 0, self.disp.width, self.disp.height), self.back_color)
        self.draw.text((self.text_margin, self.text_margin), self.header, font=self.font, fill=self.head_color)
        self.draw.text((self.text_margin, self.text_margin+2.0*self.size_y), label, font=self.font, fill=self.text_color)
        self.draw.text((self.text_margin, self.text_margin+3.5*self.size_y), value, font=self.font_big, fill=self.text_color)
        self.disp.display(self.img)

    def clear(self):
        self.draw.rectangle((0, 0, self.disp.width, self.disp.height), self.back_color)
        self.disp.display(self.img)

#### Measuring Loop

In [ ]:
lcd=lcdscreen()
max30102.sensor_start_collect()
time.sleep(1)

values = []   # List for plotting

In [ ]:
try:
    while True:  
        now = datetime.now()

        max30102.get_heartbeat_SPO2()
        h = max30102.heartbeat
        o = max30102.SPO2
        t = max30102.get_temperature_c()
        lcd.update("Heartbeat",f"{h} beats/min")
        
        values.append(h)
        
        pyplot.plot(values)
        pyplot.title(f"O2: {o} %")
        pyplot.title(f"H: {h} beats/min", loc='left')
        pyplot.title(f"T: {t:.1f} C", loc='right')
        pyplot.suptitle(f"Updated: {now.strftime('%H:%M:%S')}")
        clear_output(wait=True)
        display(pyplot.gcf())
        
        time.sleep(1)
        
except KeyboardInterrupt:
    max30102.sensor_end_collect()
    lcd.clear()
    print('Done')
